## Imports Library :

In [ ]:
import numpy as np
import pandas as pd
from __future__ import division
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from io import BytesIO
import requests
import bq_helper
from sklearn.model_selection import train_test_split
import keras.backend as K
import keras_rcnn as KC
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Convolution2D, Flatten, MaxPooling2D, Dropout, Activation, Reshape, Input
from keras.utils import to_categorical
from keras.models import load_model
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.applications.vgg16 import decode_predictions, VGG16
import tensorflow as tf
import queue as Q
import math
import random
import os

# DEFINITION OF FUNCTIONS

In [ ]:
# Return an image from its url
def images_from_url(url):
    try:
        response = requests.get(url)
        return Image.open(BytesIO(response.content))
    except:
        return False

In [ ]:
# Used to display (for an image, a list of bboxes and a list of labels) an image with labeled objects
def plot_bbox_label(image, bbox, label):
    im_dim_y = image.shape[0]
    im_dim_x = image.shape[1]
    
    plt.figure(figsize=(15,20))
    fig, ax = plt.subplots(1,figsize=(15,20))
    ax.imshow(image)
    
    it = 0
    for l_bbox in bbox:
        im_width = l_bbox[2] - l_bbox[0]
        im_height = l_bbox[3] - l_bbox[1]
        
        np.random.seed(seed = int(np.prod(bytearray(label[it], 'utf8'))) %2**32)
        color = np.random.rand(3,1)
        color = np.insert(color, 3, 0.7)
        
        ax.add_patch(patches.Rectangle((l_bbox[0]*im_dim_x, l_bbox[1]*im_dim_y), im_width*im_dim_x, im_height*im_dim_y, linewidth=8, edgecolor=color, facecolor='none'));
        text = ax.annotate(label[it], (l_bbox[2]*im_dim_x,l_bbox[1]*im_dim_y), bbox=dict(boxstyle="square,pad=0.3", fc=color, lw=2))
        text.set_fontsize(18)
        it = it+1
    plt.show()

In [ ]:
# Return the result of the querry having all the information we need
# [URL, number of objects on the image, label of the object, English label, digital label, x min, x max, y min, y max]
def query_dataset(size):
    print("Loading bbox dataset...")
    sub_query_images = """
    (SELECT image_id, thumbnail_300k_url
    FROM `bigquery-public-data.open_images.images`
    WHERE thumbnail_300k_url IS NOT NULL)"""
    
    sub_query_box = """
    (SELECT image_id, label_name, x_min, x_max, y_min, y_max
    FROM `bigquery-public-data.open_images.annotations_bbox`
    ORDER BY image_id
    LIMIT """ + str(size) + """)"""

    sub_query_occ_img = """
    (SELECT image_id, COUNT(*) AS nb
    FROM `bigquery-public-data.open_images.annotations_bbox`
    GROUP BY image_id
    ORDER BY image_id
    LIMIT """ + str(size) + """)"""
    
    sub_query_word = """
    (SELECT label_name, label_display_name
    FROM `bigquery-public-data.open_images.dict`)
    """

    sub_query_id_word = """
    (SELECT lab.label_name, ROW_NUMBER() OVER (ORDER BY lab.label_name) - 1 AS id
    FROM (SELECT DISTINCT(label_name) FROM """ + sub_query_box + """) lab)"""

    main_query = """
    SELECT img.thumbnail_300k_url, occ.nb, box.label_name, wrd.label_display_name, idw.id, box.x_min, box.x_max, box.y_min, box.y_max
    FROM """ + sub_query_box + """ box
    INNER JOIN """ + sub_query_occ_img + """ occ ON occ.image_id = box.image_id
    INNER JOIN """ + sub_query_images + """ img ON occ.image_id = img.image_id
    INNER JOIN """ + sub_query_word + """ wrd ON wrd.label_name = box.label_name
    INNER JOIN """ + sub_query_id_word + """ idw ON idw.label_name = box.label_name
    ORDER BY thumbnail_300k_url"""
    
    print("Dataset loaded")
    return open_images.query_to_pandas_safe(main_query)

In [ ]:
# Load images and bboxes / labels in the right format into the ram
# This part also contains "vestiges" to realize the train of a classifier, but we use that of VGG16 which is more powerful
def load_data(data_start, data_length):
    print("Loading images from URL... (From " + str(data_start) + " to " + str(data_start + data_length) + ")")
    
    tab_image = []
    tab_list_bbox = []
    tab_list_word = []
    tab_list_labl = []
    
    # As long as there are still images in the sub-part of our dataset
    it_tuple_image = data_start
    while it_tuple_image < (data_start + data_length):
        list_bbox = []
        list_word = []
        list_labl = []
        
        # We get the image from the link
        ulr_image = dataset.thumbnail_300k_url.loc[[it_tuple_image]].iloc[0]
        image = images_from_url(ulr_image)
        
        nb_bbox = dataset.nb.loc[[it_tuple_image]].iloc[0] # The number of bbox = the number of objects
        
        if(image != False):
            #Resize it on normalize image
            image_w, image_h = image.size
            taille_max = max(image_w, image_h)
            coef = 800/taille_max
            image = image.resize((int(coef*image_w), int(coef*image_h)))
            image = np.array(image)/255

            # We only process the images that are in RGB (This also removes the more available images)
            if(len(image.shape) == 3):
                # We insert the image in tab_image with the normalized pixel value
                tab_image.append(image)

                # For each bbox of the image, we store it in a list that we store in tab_list_bbox
                for it_bbox in range (0, nb_bbox):
                    it_tuple_bbox = it_tuple_image + it_bbox
                    if(it_tuple_bbox < data_start + data_length):
                        list_bbox.append([dataset.x_min.loc[[it_tuple_bbox]].iloc[0], dataset.y_min.loc[[it_tuple_bbox]].iloc[0], dataset.x_max.loc[[it_tuple_bbox]].iloc[0], dataset.y_max.loc[[it_tuple_bbox]].iloc[0]])

                        one_hot = np.zeros(600)
                        one_hot[dataset.id.loc[[it_tuple_bbox]].iloc[0]] = 1
                        list_word.append(one_hot)
                        
                        label = dataset.label_display_name.loc[[it_tuple_bbox]].iloc[0]
                        list_labl.append(label)

                tab_list_bbox.append(list_bbox)
                tab_list_word.append(list_word)
                tab_list_labl.append(list_labl)
            # To understand this jump you have to understand the structure of dataset_bbox
        it_tuple_image = it_tuple_image + nb_bbox
                                                               
    tab_image = np.array(tab_image)
    tab_list_bbox = np.array(tab_list_bbox)
    tab_list_word = np.array(tab_list_word)
    tab_list_labl = np.array(tab_list_labl)
    
    print("Image loaded")
    
    return [tab_image, tab_list_bbox, tab_list_word, tab_list_labl]

In [ ]:
# Calculate the IoU for 2 boxes in xmin ymin xmax ymax ymax format
def IoU(bbox1, bbox2):
    w_intersect = (bbox1[2] - bbox1[0]) + (bbox2[2] - bbox2[0]) - (max(bbox1[2], bbox2[2]) - min(bbox1[0], bbox2[0]))
    h_intersect = (bbox1[3] - bbox1[1]) + (bbox2[3] - bbox2[1]) - (max(bbox1[3], bbox2[3]) - min(bbox1[1], bbox2[1]))
    
    if(w_intersect < 0 or h_intersect < 0):
        return 0
    
    intersect = w_intersect * h_intersect

    union_1 = (bbox1[2]-bbox1[0]) * (bbox1[3]-bbox1[1])
    union_2 = (bbox2[2]-bbox2[0]) * (bbox2[3]-bbox2[1])
    
    union = union_1 + union_2 - intersect

    return intersect/union

In [ ]:
# Create anchors with the center (x, y) and the width / height of the convolution (reduction 16)
def generate_anchors(center_x, center_y, conv_w, conv_h):
    anchor_ratio = [[1, 1], [1, 2], [2, 1]]
    anchor_coef = [1, 2, 4]
    anchor_size = 128
    
    anchor_list = []
    
    for ratio in anchor_ratio:
        for coef in anchor_coef:
            anchor_width = (anchor_size*coef*ratio[0]) / (conv_w*16)
            anchor_height = (anchor_size*coef*ratio[1]) / (conv_h*16)
            anchor_x = (center_x/conv_w) - (anchor_width/2)
            anchor_y = (center_y/conv_h) - (anchor_height/2)
            anchor = [anchor_x, anchor_y, anchor_x+anchor_width, anchor_y+anchor_height]
            
            anchor_list.append(anchor)
    
    anchor_list = np.array(anchor_list)
    
    return anchor_list

In [ ]:
# RoI pooling operation on an array and shape*shape size
# There are 3 similar lines depending on the type of operation used for pooling (after testing the average is even the max)
def RoI(array, shape):
    result = np.zeros((shape, shape, array.shape[2]))
    for i in range (0, shape):
        for j in range (0, shape):
            sub_array = array[int(i*array.shape[0]/shape):int((i+1)*array.shape[0]/shape), int(j*array.shape[1]/shape):int((j+1)*array.shape[1]/shape)]
            #result[i][j] = np.amax(np.amax(sub_array, axis = 0), axis = 0)
            result[i][j] = np.mean(np.mean(sub_array, axis = 0), axis = 0)
            #result[i][j] = np.amin(np.amin(sub_array, axis = 0), axis = 0)
    return result

In [ ]:
# Extracts the convolution part of VGG16
def generate_conv():
    vgg16_net = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
    model = Model(input=vgg16_net.layers[0].input, output=vgg16_net.layers[17].output)
    return model

In [ ]:
# A custom accuracy, it is slightly capable of exceeding 1 but otherwise with keras,
# For a custom loss, the accuracy is buggy (it is a known bug)
def acc(y_true, y_pred): return K.mean(K.round(y_pred)*y_true + (1.-K.round(y_pred))*(1.-y_true))

In [ ]:
# Loss custom to classify it from the rpn, we igniore the cases where the prediction is (0, 0) and we strengthen the learning when the presence of an object
def custom_loss_rpn_cls(y_true, y_pred):
    shape = K.shape(y_true)
    depth = shape[0]
        
    # Removes [0, 0] (i.e. between object and no object)
    new_y_pred = K.zeros((depth, 2))
    for i in range (0, 9):
        cond = K.equal(y_true[:, 2*i:2*i+2], [0,0])
        cond = tf.math.logical_and(cond[:,0], cond[:,1])
        cond = K.concatenate([cond, cond])
        cond = K.reshape(cond, (depth,2))
        
        temp = K.switch(cond, y_true[:, 2*i:2*i+2], y_pred[:, 2*i:2*i+2])
        if i == 0:
            new_y_pred = temp
        else:
            new_y_pred = K.concatenate([new_y_pred, temp])

    new_y_pred = K.reshape(new_y_pred, (depth, 18))
    cls = K.binary_crossentropy(y_true, new_y_pred)
    
    # Reinforces the [1,0] (i.e. the presence of object)
    new_cls = K.zeros((depth, 2))
    for i in range (0, 9):
        cond = K.equal(y_true[:, 2*i:2*i+2], [0,1])
        cond = tf.math.logical_and(cond[:,0], cond[:,1])
        cond = K.concatenate([cond, cond])
        cond = K.reshape(cond, (depth,2))
        
        temp = K.switch(cond, cls[:, 2*i:2*i+2], cls[:, 2*i:2*i+2]*4.7)
        if i == 0:
            new_cls = temp
        else:
            new_cls = K.concatenate([new_cls, temp])
    new_cls = K.reshape(new_cls, (depth, 18))
    
    # We multiply again to compensate for the [0,0]
    # The coeficients were found experimentally
    return K.mean(new_cls*2.1)

In [ ]:
# Return the model used to classify it from the rpn
def generate_rpn_cls():
    model = Sequential()
    
    model.add(Flatten(input_shape=(3, 3, 512)))
    
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))

    model.add(Dense(18))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.4))

    model.compile(loss=custom_loss_rpn_cls, optimizer='adam', metrics=[acc])
    
    return model

In [ ]:
# smoothL1 function (known function)
def smoothL1(y_true, y_pred):
    x   = K.abs(y_true - y_pred)
    x   = K.switch(x < 1, x*x, x)
    return  x

In [ ]:
# Loss custom for the regresseur of the rpn, we igniore the cases where the prediction is (0, 0)
def custom_loss_rpn_reg(y_true, y_pred):
    shape = K.shape(y_true)
    depth = shape[0]
        
    # Removes the [0, 0, 0, 0] i.e. there is no presence of object
    new_y_pred = K.zeros((depth, 4))
    for i in range (0, 9):
        cond = K.equal(y_true[:, 2*i:2*i+4], [0,0,0,0])
        cond = tf.math.logical_and(tf.math.logical_and(cond[:,0], cond[:,1]), tf.math.logical_and(cond[:,2], cond[:,3]))
        cond = K.concatenate([cond, cond, cond, cond])
        cond = K.reshape(cond, (depth,4))
        
        temp = K.switch(cond, y_true[:, 2*i:2*i+4], y_pred[:, 2*i:2*i+4])
        if i == 0:
            new_y_pred = temp
        else:
            new_y_pred = K.concatenate([new_y_pred, temp])

    new_y_pred = K.reshape(new_y_pred, (depth, 36))
    reg = smoothL1(y_true, new_y_pred)
    
    # We multiply again to compensate for the [0,0,0,0]
    # The coeficients were found experimentally
    return K.mean(reg)*9

In [ ]:
# Return the model used for the rpn regresser
def generate_rpn_reg():
    model = Sequential()
    
    model.add(Flatten(input_shape=(3, 3, 512)))
    
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))

    model.add(Dense(36))
    model.add(Activation('linear'))
    model.add(Dropout(0.4))

    model.compile(loss=custom_loss_rpn_reg, optimizer='adam', metrics=[acc])
    
    return model

In [ ]:
# Return the model used to classify it
def generate_cls_nn():
    vgg16_net = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
    
    l_input = Input(shape=(7, 7, 512))
    l_flatten = vgg16_net.get_layer("flatten")
    l_fc1 = vgg16_net.get_layer("fc1")
    l_fc2 = vgg16_net.get_layer("fc2")
    l_output = vgg16_net.get_layer("predictions")
    model = Model(input=l_input, output=l_output(l_fc2(l_fc1(l_flatten(l_input)))))
    return model

In [ ]:
# Generate the data to make it compatible with the networks of the rpn
def generate_feature_label_rpn():
    features = []
    labels = []
    
    nb_feature_map = list_feature_map.shape[0]
    
    nb_max = []
    for i in range(0,9):
        nb_max.append(0)
    
    # For each 
    for it_feature_map in range (0, nb_feature_map):
        feature_map = list_feature_map[it_feature_map]
        
        feature_map_width = feature_map.shape[1]
        feature_map_height = feature_map.shape[0]
        
        # Chaque sliding window
        for x in range (1, feature_map_width - 1):
            for y in range (1, feature_map_height - 1):
                
                sub_labels = []
                sub_anchor = []
                
                window_valid = False
                
                list_anchors = generate_anchors(x, y, feature_map_width, feature_map_height)
                
                it_anch = -1
                good_it_anch = it_anch
                
                # Chaque anchors
                for anchor in list_anchors:
                    it_anch = it_anch + 1
                    
                    anchor_cross = not(anchor[0] >= 0 and anchor[1] >= 0 and anchor[0] + anchor[2] < 1 and anchor[1] + anchor[3] < 1)
                    anchor_cross = False
                    anchor_valid = False
                    anchor_empty = True

                    # If the anchor is > 0.7 , or > 0.3 once
                    for bbox in data_bbox[it_feature_map]:
                        if(IoU(anchor, bbox) > 0.7):
                            anchor_valid = True
                            window_valid = True
                            break
                        if(IoU(anchor, bbox) > 0.3):
                            anchor_empty = False
                    
                    # Valid anchor case
                    if(anchor_valid and not(anchor_cross)):
                        good_it_anch = it_anch
                        sub_labels.append(1.)
                        sub_labels.append(0.)
                        
                        # Coordinates part relating to the anchor 
                        bbox_x, bbox_y, bbox_xm, bbox_ym = bbox
                        bbox_width = bbox_xm - bbox_x
                        bbox_height = bbox_ym - bbox_y
                        anchor_x, anchor_y, anchor_xm, anchor_ym = anchor
                        anchor_width = anchor_xm - anchor_x
                        anchor_height = anchor_ym - anchor_y
                        
                        sub_anchor.append((bbox_x - anchor_x)/anchor_width)
                        sub_anchor.append((bbox_y - anchor_y)/anchor_height)
                        sub_anchor.append(math.log(bbox_width/anchor_width))
                        sub_anchor.append(math.log(bbox_height/anchor_height))
                    # Empty anchor case
                    elif(anchor_empty and not(anchor_cross)):
                        sub_labels.append(0.)
                        sub_labels.append(1.)

                        sub_anchor.append(0.)
                        sub_anchor.append(0.)
                        sub_anchor.append(0.)
                        sub_anchor.append(0.)
                    # Case anchor between the 2
                    else:
                        sub_labels.append(0.)
                        sub_labels.append(0.)

                        sub_anchor.append(0.)
                        sub_anchor.append(0.)
                        sub_anchor.append(0.)
                        sub_anchor.append(0.)
                
                # Normalization (that the window with one object and no more than 400 object per anchor (400 is a number found experimentally))
                if(window_valid and nb_max[good_it_anch] < 400):
                    nb_max[good_it_anch] = nb_max[good_it_anch] + 1
                    
                    features.append(feature_map[y-1:y+2, x-1:x+2])
                    labels.append(np.array(sub_labels + sub_anchor))
            
    features = np.array(features)
    labels = np.array(labels)

    return features, labels

In [ ]:
# Vestige of our classifer (the one that replaces vgg)
def generate_feature_label_cls():
    features = []
    labels = []
    
    counter = 0
    
    nb_image_conv = data_conv.shape[0]
    for it_image_conv in range (0, nb_image_conv):
        for it_bbox in range (0, len(data_bbox[it_image_conv])):
            list_bbox = data_bbox[it_image_conv]
            bbox = list_bbox[it_bbox]
            features.append(add_black_border(data_conv[it_image_conv][int(bbox[0]*13):int(bbox[2]*13), int(bbox[1]*13):int(bbox[3]*13)]))
            labels.append(data_word[it_image_conv][it_bbox])
            
    features = np.array(features)
    labels = np.array(labels)
    
    return features, labels

In [ ]:
# Return the convolution of an image
def pred_conv(image):
    return conv_net.predict(np.array([image]))[0]

# DEFINITION OF THE GLOBAL

In [ ]:
# Variable dataset
open_images = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="open_images")
query_size = 100000
dataset_size = 1000
dataset = query_dataset(query_size)

In [ ]:
conv_net = generate_conv()

In [ ]:
rpn_cls_net = generate_rpn_cls()

In [ ]:
rpn_reg_net = generate_rpn_reg()

In [ ]:
cls_net = generate_cls_nn()

# TRAIN PART

In [ ]:
# Loads images from 0 to dataset_size (1000) in ram, as well as their label
data_image, data_bbox, data_word, data_labl = load_data(0, dataset_size)
del dataset

In [ ]:
# Turns images into convolution
list_feature_map = np.array(list(map(pred_conv, data_image)))
del data_image

In [ ]:
# Generate the data to make it compatible with the networks of the rpn
# (the data is also mixed and a validation part is extracted)
features_rpn, labels_rpn = generate_feature_label_rpn()
X_train_rpn, X_valid_rpn, y_train_rpn, y_valid_rpn = train_test_split(features_rpn, labels_rpn, test_size=0.1, random_state=42)
del features_rpn
del labels_rpn

In [ ]:
# Learning the network "regression" of rpn
rpn_reg_net.fit(X_train_rpn, y_train_rpn[:, 18:54], validation_data=(X_valid_rpn, y_valid_rpn[:, 18:54]), epochs=100, batch_size=32)

In [ ]:
# Learning the rpn "classify" network
rpn_cls_net.fit(X_train_rpn, y_train_rpn[:, 0:18], validation_data=(X_valid_rpn, y_valid_rpn[:, 0:18]), epochs=2000, batch_size=64)

In [ ]:
# We save the networks of the rpn to be able to use them without train
rpn_reg_net.save("reg.h5")
rpn_cls_net.save("cls.h5")

In [ ]:
from sklearn.externals import joblib

joblib_file = "export.pkl"   
joblib.dump(rpn_cls_net, joblib_file)

# from sklearn.externals import joblib
# loaded_model = joblib.load('best_model.pkl')

# APPLICATION PART

In [ ]:
def predict(URL, threshold_cls_rpn = 0.7, threshold_cls_vgg = 0.5, threshold_iou = 0.8):
    # The image is downloaded and put in the right format (the vgg format is not normalized, that of our rpn is because gives us better results)
    image_test = images_from_url(URL)
    if(image_test != False):
        image_test_w, image_test_h = image_test.size
        taille_max = max(image_test_w, image_test_h)
        coef = 800/taille_max
        image_test = image_test.resize((int(coef*image_test_w), int(coef*image_test_h)))
        image_test_vgg = np.array(image_test)
        image_test = np.array(image_test)/255
        if(len(image_test.shape) == 3):
            anchor_test_valid = []

            image_test_conv_vgg = pred_conv(image_test_vgg)
            image_test_conv = pred_conv(image_test)

            # We pass on each pixel of the convolution
            for x in range (1, image_test_conv.shape[1] - 1):
                for y in range (1, image_test_conv.shape[0] - 1):
                    anchor_valid = False
                    anchor_empty = True

                    # We make a prediction on the sliding window centered on the current pixel
                    pred_cls = rpn_cls_net.predict(np.array([image_test_conv[y-1:y+2, x-1:x+2]]))[0]
                    pred_reg = rpn_reg_net.predict(np.array([image_test_conv[y-1:y+2, x-1:x+2]]))[0]

                    # On test for all anchors if the rpn has detected an object
                    list_anchors = generate_anchors(x, y, image_test_conv.shape[1], image_test_conv.shape[0])
                    for k in range(0, 9):
                        # If we find an object with more than 70% safety
                        if(pred_cls[k*2] >= threshold_cls_rpn):
                            # We recover the information of the anchor
                            anchor = list_anchors[k]
                            anchor_x, anchor_y, anchor_xm, anchor_ym = anchor
                            anchor_width = anchor_xm - anchor_x
                            anchor_height = anchor_ym - anchor_y

                            # We retrieve the info of the prediction
                            pred_reg_x, pred_reg_y, pred_reg_w, pred_reg_h = pred_reg[k*4:k*4+4]

                            # We test if the anchor does not come out of the screen
                            cond1 = anchor_x+(pred_reg_x*anchor_width) >= 0
                            cond2 = anchor_y+(pred_reg_y*anchor_height) >= 0
                            cond3 = anchor_x+(pred_reg_x*anchor_width) + (10**pred_reg_w)*anchor_width < 1
                            cond4 = anchor_y+(pred_reg_y*anchor_height) + (10**pred_reg_h)*anchor_height < 1
                            if(cond1 and cond2 and cond3 and cond4):
                                # We calculate the xmin/max ymin/max of the prediction relative to the image
                                it_min_x = int((anchor_x+(pred_reg_x*anchor_width)) * image_test_conv.shape[1])
                                it_max_x = int((anchor_x+(pred_reg_x*anchor_width) + (10**pred_reg_w)*anchor_width) * image_test_conv.shape[1])
                                it_min_y = int((anchor_y+(pred_reg_y*anchor_height)) * image_test_conv.shape[0])
                                it_max_y = int((anchor_y+(pred_reg_y*anchor_height) + (10**pred_reg_h)*anchor_height) * image_test_conv.shape[0])

                                # If the prediction is wider than 7*7 (minimum of the classifier)
                                if(it_max_y-it_min_y >= 7 and it_max_x-it_min_x >= 7):
                                    # Prediction of the first 5 classes that vgg finds
                                    label = decode_predictions(cls_net.predict(np.array([RoI(image_test_conv_vgg[it_min_y:it_max_y, it_min_x:it_max_x], 7)])), top=5)[0]
                                    # If the trust given to the top class of vgg is more than 50%
                                    if(label[0][2] >= threshold_cls_vgg):
                                        # We store the data at the simplest to process them with the nonmax
                                        anchor_test_valid.append([label[0][2], [[label[0][1], label[1][1], label[2][1], label[3][1], label[4][1]], anchor_x+(pred_reg_x*anchor_width), anchor_y+(pred_reg_y*anchor_height), anchor_x+(pred_reg_x*anchor_width) + (10**pred_reg_w)*anchor_width, anchor_y+(pred_reg_y*anchor_height) + (10**pred_reg_h)*anchor_height]])  

            # SECTION NONMAX
            # The goal is to remove overlaps above a threshold for the same classes (all it takes is a coresspondance in the first 5 classes)
            anchor_test_valid = np.array(anchor_test_valid)      
            q = Q.PriorityQueue()
            for a in anchor_test_valid:
                q.put((1-a[0] + random.random()/100000,a[1]))
            anchor_test_valid = []
            size = q.qsize()
            for i in range (0, size):
                var_i = q.get()
                found_one = False
                for a in anchor_test_valid:
                    for labelnb in range(0, 5):
                        if(IoU(a[1], var_i[1][1:5]) >= 1 - threshold_iou and a[0] == var_i[1][0][labelnb]):
                            found_one = True
                if(not found_one):
                    anchor_test_valid.append([var_i[1][0][0], var_i[1][1:5]])
            anchor_test_valid = np.array(anchor_test_valid)
            if(anchor_test_valid.shape[0] != 0):
                plot_bbox_label(image_test, anchor_test_valid[:, 1], anchor_test_valid[:, 0])
            else:
                plt.figure(figsize=(15,20))
                plt.imshow(image_test_vgg)
                plt.show()

In [ ]:
def predict_multiple(list_URL):
    for URL in list_URL:
        url_pred = predict(URL)

In [ ]:
# A CAR :   https://www.usinenouvelle.com/mediatheque/4/5/4/000626454_image_896x598/dacia-sandero.jpg
# 2 CARS :   https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/s17-2051-fine-1553003760.jpg
# N CARS :   https://cdn-images-1.medium.com/max/1600/1*ICvAO8mPCA_sXOzW9zeM7g.jpeg
# 2 CARS :   https://ischool.syr.edu/infospace/wp-content/files/2015/10/toyota-and-lexus-car-on-road--e1444655872784.jpg
# ZOO : http://www.mdjunited.com/medias/images/zoo.jpg

url_images_test = ['https://www.usinenouvelle.com/mediatheque/4/5/4/000626454_image_896x598/dacia-sandero.jpg',
                   'https://images5.alphacoders.com/393/393962.jpg',
                   'https://images.unsplash.com/photo-1544776527-68e63addedf7?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80',
                   'https://www.autocar.co.uk/sites/autocar.co.uk/files/styles/gallery_slide/public/images/car-reviews/first-drives/legacy/gallardo-0638.jpg?itok=-So1NoXA', 
                   'http://www.mdjunited.com/medias/images/zoo.jpg']

predict_multiple(url_images_test)